## Exploratory Data Analysis of Lending Club Dataset
** Note - All cells are collapsed to avoid longer output screen. The notebook is basically practised on Databricks community edition.

In [ ]:
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
from pyspark.sql.types import *

In [ ]:
%sql
select * from processes_loan_ver2 limit 10

term,home_ownership,grade,purpose,int_rate,addr_state,loan_status,application_type,loan_amnt,emp_length,annual_inc,dti,delinq_2yrs,revol_util,revol_bal,total_acc,num_tl_90g_dpd_24m,dti_joint,emp_cleaned,term_cleaned,revol_util_cleaned,revol_util_avg,dti_cleaned,bad_loan
36 months,RENT,G,other,30.84%,WA,Current,Individual,1000,6 years,31680.0,0.0,0,null,0,3,0,null,6,36,43.76206961077844,43.76206961077844,0.0,No
60 months,MORTGAGE,B,debt_consolidation,12.98%,NJ,Current,Individual,30000,10+ years,137000.0,12.87,0,65.20%,23162,31,0,null,10,60,65.0,43.76206961077844,12.87,No
60 months,MORTGAGE,A,credit_card,8.19%,MD,Current,Individual,17000,6 years,82000.0,15.57,0,25.80%,16693,17,0,null,6,60,25.0,43.76206961077844,15.57,No
36 months,RENT,B,debt_consolidation,10.72%,NJ,Current,Individual,3000,2 years,60000.0,12.46,1,39.10%,586,16,0,null,2,36,39.0,43.76206961077844,12.46,No
36 months,RENT,B,debt_consolidation,11.31%,IL,Current,Joint App,24000,2 years,74675.0,17.28,0,44%,18469,9,0,13.88,2,36,44.0,43.76206961077844,17.28,No
36 months,MORTGAGE,E,medical,23.40%,TX,Current,Individual,5000,10+ years,75000.0,8.16,0,51.80%,16274,13,0,null,10,36,51.0,43.76206961077844,8.16,No
36 months,MORTGAGE,A,debt_consolidation,8.19%,MI,Current,Individual,26000,n/a,88000.0,33.92,0,56.10%,8698,57,0,null,,36,56.0,43.76206961077844,33.92,No
36 months,MORTGAGE,B,other,11.80%,CA,Current,Individual,5000,10+ years,100000.0,17.88,2,17.20%,4093,29,0,null,10,36,17.0,43.76206961077844,17.88,No
36 months,RENT,C,debt_consolidation,16.14%,VA,Current,Individual,12400,n/a,25764.0,38.33,0,75.30%,10315,11,0,null,,36,75.0,43.76206961077844,38.33,No
60 months,MORTGAGE,C,debt_consolidation,16.14%,DE,Current,Individual,40000,10+ years,390000.0,9.14,0,73%,57115,28,0,null,10,60,73.0,43.76206961077844,9.14,No


In [ ]:
loan_data = spark.table('processes_loan_ver2')
display(loan_data)

term,home_ownership,grade,purpose,int_rate,addr_state,loan_status,application_type,loan_amnt,emp_length,annual_inc,dti,delinq_2yrs,revol_util,revol_bal,total_acc,num_tl_90g_dpd_24m,dti_joint,emp_cleaned,term_cleaned,revol_util_cleaned,revol_util_avg,dti_cleaned,bad_loan
36 months,RENT,G,other,30.84%,WA,Current,Individual,1000,6 years,31680.0,0.0,0,null,0,3,0,null,6,36,43.76206961077844,43.76206961077844,0.0,No
60 months,MORTGAGE,B,debt_consolidation,12.98%,NJ,Current,Individual,30000,10+ years,137000.0,12.87,0,65.20%,23162,31,0,null,10,60,65.0,43.76206961077844,12.87,No
60 months,MORTGAGE,A,credit_card,8.19%,MD,Current,Individual,17000,6 years,82000.0,15.57,0,25.80%,16693,17,0,null,6,60,25.0,43.76206961077844,15.57,No
36 months,RENT,B,debt_consolidation,10.72%,NJ,Current,Individual,3000,2 years,60000.0,12.46,1,39.10%,586,16,0,null,2,36,39.0,43.76206961077844,12.46,No
36 months,RENT,B,debt_consolidation,11.31%,IL,Current,Joint App,24000,2 years,74675.0,17.28,0,44%,18469,9,0,13.88,2,36,44.0,43.76206961077844,17.28,No
36 months,MORTGAGE,E,medical,23.40%,TX,Current,Individual,5000,10+ years,75000.0,8.16,0,51.80%,16274,13,0,null,10,36,51.0,43.76206961077844,8.16,No
36 months,MORTGAGE,A,debt_consolidation,8.19%,MI,Current,Individual,26000,n/a,88000.0,33.92,0,56.10%,8698,57,0,null,,36,56.0,43.76206961077844,33.92,No
36 months,MORTGAGE,B,other,11.80%,CA,Current,Individual,5000,10+ years,100000.0,17.88,2,17.20%,4093,29,0,null,10,36,17.0,43.76206961077844,17.88,No
36 months,RENT,C,debt_consolidation,16.14%,VA,Current,Individual,12400,n/a,25764.0,38.33,0,75.30%,10315,11,0,null,,36,75.0,43.76206961077844,38.33,No
60 months,MORTGAGE,C,debt_consolidation,16.14%,DE,Current,Individual,40000,10+ years,390000.0,9.14,0,73%,57115,28,0,null,10,60,73.0,43.76206961077844,9.14,No


In [ ]:
display(loan_data.describe())

summary,term,home_ownership,grade,purpose,int_rate,addr_state,loan_status,application_type,loan_amnt,emp_length,annual_inc,dti,delinq_2yrs,revol_util,revol_bal,total_acc,num_tl_90g_dpd_24m,dti_joint,emp_cleaned,term_cleaned,revol_util_cleaned,revol_util_avg,dti_cleaned,bad_loan
count,128412,128412,128412,128412,128412,128412,128412,128412,128412,128412,128412,128175,128412,128256,128412,128412,128412,16782,128412,128412,128412,128412,128412,128412
mean,null,null,null,null,null,null,null,null,15971.32102139987,null,82797.32786094758,19.9331775307197,0.22783696227766875,null,16898.000957854405,22.677413325857398,0.059488209824626985,19.226602312000953,5.609821091955992,43.51948416035884,43.76206961077843,43.7620696107706,19.930878500451655,null
stddev,null,null,null,null,null,null,null,null,10150.384232741908,null,108298.46579150062,20.14354224347552,0.7337929617806055,null,24082.550912035458,12.129215673024765,0.4106517365944237,8.14163126459557,3.6776295739511053,11.132202965983407,24.786779696453944,0.0,20.128079702803714,null
min,36 months,ANY,A,car,10.08%,AK,Charged Off,Individual,1000,1 year,0.0,0.0,0,0%,0,2,0,0.0,,36,0.0,43.76206961077844,0.0,No
max,60 months,RENT,G,vacation,8.81%,WY,Late (31-120 days),Joint App,40000,n/a,9757200.0,999.0,24,99.90%,2358150,160,23,39.99,9,60,183.0,43.76206961077844,999.0,Yes


In [ ]:
#Check which state has more customers.
loan_data.groupBy('addr_state').count().orderBy(desc('count')).show(5)
#California, Texas and Ney York the top 3 states.

+----------+-----+
addr_state|count|
+----------+-----+
 CA|17879|
 TX|10508|
 NY|10112|
 FL| 9758|
 IL| 4963|
+----------+-----+
only showing top 5 rows

In [ ]:
# Checking which state has most number of 'bad_loans'
display(loan_data.filter(loan_data.bad_loan=='Yes').groupBy('addr_state').count().orderBy(desc('count')))
#Surprising - Florida has more bad loans though it is no on the top 3 list of consumers

addr_state,count
CA,284
NY,177
FL,153
TX,146
PA,72
GA,60
MD,60
NC,55
VA,54
MI,51


In [ ]:
grade_data = loan_data.groupBy("grade").agg(sum('loan_amnt')).orderBy(desc('sum(loan_amnt)'))
display(grade_data)

grade,sum(loan_amnt)
A,604816200
B,591596125
C,495916925
D,267034700
E,83832000
F,6326275
G,1387050


In [ ]:
#Among each grade how much loan amount is accumulated in the bad_loan
display(loan_data.groupBy('grade', 'bad_loan').agg(sum('loan_amnt')))
#The graph shows that loans under G grade have high chances of defaulters because 13% of it fall under bad loan. This scenario improves from G through A.

grade,bad_loan,sum(loan_amnt)
G,Yes,176475
B,No,584562525
F,No,5842125
E,Yes,3624800
C,Yes,8643900
B,Yes,7033600
A,Yes,3148050
F,Yes,484150
C,No,487273025
A,No,601668150


In [ ]:
# Create an exposure column to indicate weigh the revol_balance based on the bad loan. If the loan is bad loan then we multiply it with -10, where
# - is to show that it is bad loan and 10 to weigh the intensity of it for revol_bal.
exposure_data = loan_data.withColumn('exposure',when(loan_data.bad_loan=='No',col('revol_bal')).otherwise(-10*col('revol_bal')))
display(exposure_data)

term,home_ownership,grade,purpose,int_rate,addr_state,loan_status,application_type,loan_amnt,emp_length,annual_inc,dti,delinq_2yrs,revol_util,revol_bal,total_acc,num_tl_90g_dpd_24m,dti_joint,emp_cleaned,term_cleaned,revol_util_cleaned,revol_util_avg,dti_cleaned,bad_loan,exposure
36 months,RENT,G,other,30.84%,WA,Current,Individual,1000,6 years,31680.0,0.0,0,null,0,3,0,null,6,36,43.76206961077844,43.76206961077844,0.0,No,0
60 months,MORTGAGE,B,debt_consolidation,12.98%,NJ,Current,Individual,30000,10+ years,137000.0,12.87,0,65.20%,23162,31,0,null,10,60,65.0,43.76206961077844,12.87,No,23162
60 months,MORTGAGE,A,credit_card,8.19%,MD,Current,Individual,17000,6 years,82000.0,15.57,0,25.80%,16693,17,0,null,6,60,25.0,43.76206961077844,15.57,No,16693
36 months,RENT,B,debt_consolidation,10.72%,NJ,Current,Individual,3000,2 years,60000.0,12.46,1,39.10%,586,16,0,null,2,36,39.0,43.76206961077844,12.46,No,586
36 months,RENT,B,debt_consolidation,11.31%,IL,Current,Joint App,24000,2 years,74675.0,17.28,0,44%,18469,9,0,13.88,2,36,44.0,43.76206961077844,17.28,No,18469
36 months,MORTGAGE,E,medical,23.40%,TX,Current,Individual,5000,10+ years,75000.0,8.16,0,51.80%,16274,13,0,null,10,36,51.0,43.76206961077844,8.16,No,16274
36 months,MORTGAGE,A,debt_consolidation,8.19%,MI,Current,Individual,26000,n/a,88000.0,33.92,0,56.10%,8698,57,0,null,,36,56.0,43.76206961077844,33.92,No,8698
36 months,MORTGAGE,B,other,11.80%,CA,Current,Individual,5000,10+ years,100000.0,17.88,2,17.20%,4093,29,0,null,10,36,17.0,43.76206961077844,17.88,No,4093
36 months,RENT,C,debt_consolidation,16.14%,VA,Current,Individual,12400,n/a,25764.0,38.33,0,75.30%,10315,11,0,null,,36,75.0,43.76206961077844,38.33,No,10315
60 months,MORTGAGE,C,debt_consolidation,16.14%,DE,Current,Individual,40000,10+ years,390000.0,9.14,0,73%,57115,28,0,null,10,60,73.0,43.76206961077844,9.14,No,57115


In [ ]:
display(exposure_data.groupBy('grade', 'bad_loan').agg(sum('exposure')))
# Inference - Even though G grade loan have many defaulters it is regarded with low risk under exposure when the amount of revol_bal is taken in account.
# Where as C category loans are under higher risk of loss because the revol_bal of the customers is high.

grade,bad_loan,sum(exposure)
G,Yes,-1907150
B,No,602056854
F,No,5799837
E,Yes,-37537880
C,Yes,-81097340
B,Yes,-68627150
A,Yes,-36651160
F,Yes,-4260150
C,No,498293640
A,No,678252010


In [ ]:
# Analysing the outliers from the loan amount data
display(spark.sql('''
          select loan_amnt from processes_loan_ver2
          '''))

loan_amnt
1000
30000
17000
3000
24000
5000
26000
5000
12400
40000


In [ ]:
#Checking the distribution of the loan amount
display(spark.sql('''
          select loan_amnt from processes_loan_ver2
          '''))
# Inference - Not exactly normal distribution as the data points lie around the normal line and not on the line.

loan_amnt
1000
30000
17000
3000
24000
5000
26000
5000
12400
40000


In [ ]:
strip_percent = udf(lambda x: x.strip('%'))
loan_data = loan_data.withColumn('int_rate_clean', strip_percent(loan_data.int_rate))
loan_data = loan_data.withColumn('int_rate_clean', col('int_rate_clean').cast('float'))
loan_data.show(5)

+----------+--------------+-----+------------------+--------+----------+-----------+----------------+---------+----------+----------+-----+-----------+----------+---------+---------+------------------+---------+-----------+------------+------------------+-----------------+-----------+--------+--------------+
 term|home_ownership|grade| purpose|int_rate|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc| dti|delinq_2yrs|revol_util|revol_bal|total_acc|num_tl_90g_dpd_24m|dti_joint|emp_cleaned|term_cleaned|revol_util_cleaned| revol_util_avg|dti_cleaned|bad_loan|int_rate_clean|
+----------+--------------+-----+------------------+--------+----------+-----------+----------------+---------+----------+----------+-----+-----------+----------+---------+---------+------------------+---------+-----------+------------+------------------+-----------------+-----------+--------+--------------+
 36 months| RENT| G| other| 30.84%| WA| Current| Individual| 1000| 6 years| 31680.0| 0.0| 0| null| 0| 3| 0| null| 6| 36 | 43.76206961077844|43.76206961077844| 0.0| No| 30.84|
 60 months| MORTGAGE| B|debt_consolidation| 12.98%| NJ| Current| Individual| 30000| 10+ years| 137000.0|12.87| 0| 65.20%| 23162| 31| 0| null| 10| 60 | 65.0|43.76206961077844| 12.87| No| 12.98|
 60 months| MORTGAGE| A| credit_card| 8.19%| MD| Current| Individual| 17000| 6 years| 82000.0|15.57| 0| 25.80%| 16693| 17| 0| null| 6| 60 | 25.0|43.76206961077844| 15.57| No| 8.19|
 36 months| RENT| B|debt_consolidation| 10.72%| NJ| Current| Individual| 3000| 2 years| 60000.0|12.46| 1| 39.10%| 586| 16| 0| null| 2| 36 | 39.0|43.76206961077844| 12.46| No| 10.72|
 36 months| RENT| B|debt_consolidation| 11.31%| IL| Current| Joint App| 24000| 2 years| 74675.0|17.28| 0| 44%| 18469| 9| 0| 13.88| 2| 36 | 44.0|43.76206961077844| 17.28| No| 11.31|
+----------+--------------+-----+------------------+--------+----------+-----------+----------------+---------+----------+----------+-----+-----------+----------+---------+---------+------------------+---------+-----------+------------+------------------+-----------------+-----------+--------+--------------+
only showing top 5 rows

In [ ]:
loan_data.printSchema()

root
-- term: string (nullable = true)
-- home_ownership: string (nullable = true)
-- grade: string (nullable = true)
-- purpose: string (nullable = true)
-- int_rate: string (nullable = true)
-- addr_state: string (nullable = true)
-- loan_status: string (nullable = true)
-- application_type: string (nullable = true)
-- loan_amnt: integer (nullable = true)
-- emp_length: string (nullable = true)
-- annual_inc: double (nullable = true)
-- dti: double (nullable = true)
-- delinq_2yrs: integer (nullable = true)
-- revol_util: string (nullable = true)
-- revol_bal: integer (nullable = true)
-- total_acc: integer (nullable = true)
-- num_tl_90g_dpd_24m: integer (nullable = true)
-- dti_joint: double (nullable = true)
-- emp_cleaned: string (nullable = true)
-- term_cleaned: string (nullable = true)
-- revol_util_cleaned: double (nullable = true)
-- revol_util_avg: double (nullable = true)
-- dti_cleaned: double (nullable = true)
-- bad_loan: string (nullable = true)
-- int_rate_clean: float (nullable = true)

In [ ]:
#Analyse the average interest rate based on the bad_loan
display(loan_data.groupBy('bad_loan').agg(avg('int_rate_clean')))

bad_loan,avg(int_rate_clean)
No,12.878821221256365
Yes,16.50690922695385


In [ ]:
display(loan_data.select(['home_ownership', 'bad_loan', 'loan_amnt']))
#Distribution of the loan amount varies according to the type of home ownership. The risk of bad loan is minimum along in the category of RENT with around 11k loan amnt.
#Lesser the monthly installment in rent home ownership less probable they are defaulter.

home_ownership,bad_loan,loan_amnt
RENT,No,1000
MORTGAGE,No,30000
MORTGAGE,No,17000
RENT,No,3000
RENT,No,24000
MORTGAGE,No,5000
MORTGAGE,No,26000
MORTGAGE,No,5000
RENT,No,12400
MORTGAGE,No,40000


In [ ]:
display(loan_data.groupBy('grade', 'purpose').count())
#Majority of the reasons that customers approach lending club accross all the grades of loan is debt_consolidation and credit card.

grade,purpose,count
B,moving,156
D,credit_card,2849
F,home_improvement,30
F,moving,6
D,renewable_energy,9
C,renewable_energy,13
A,credit_card,14177
E,renewable_energy,4
F,house,7
A,medical,314


In [ ]:
display(loan_data.groupBy('grade', 'loan_status').agg(count('loan_status')))

grade,loan_status,count(loan_status)
E,Fully Paid,360
B,Late (16-30 days),78
A,In Grace Period,74
C,Late (31-120 days),234
F,Late (31-120 days),14
C,Fully Paid,1175
E,Late (16-30 days),46
F,Fully Paid,36
E,In Grace Period,54
F,Late (16-30 days),9
